In [ ]:
!pip install html2text
!pip install transformers==3.0.2
!pip install -U imbalanced-learn

     |████████████████████████████████| 778kB 19.3MB/s 
     |████████████████████████████████| 3.0MB 49.9MB/s 
     |████████████████████████████████| 890kB 50.2MB/s 
     |████████████████████████████████| 1.2MB 45.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=b14c5115b427b734bf3f02c2bb7bc47ec773eb6378a33e30bf67532c24fd5121
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 215kB 14.8MB/s 
     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import nltk
import pandas as pd 
from html2text import html2text
import re
import warnings
import itertools
import torch
from tqdm.notebook import tqdm
import chardet

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import f1_score, classification_report
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
if torch.cuda.is_available():  
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
raw_prodcuts1 = pd.read_csv(r"/content/drive/MyDrive/Veri/Migros/products1Yeni.csv", encoding="utf-8", sep =";")
raw_prodcuts2 = pd.read_csv(r"/content/drive/MyDrive/Veri/Migros/products2Yeni.csv", encoding="utf-8", sep =";")
raw_prodcuts3 = pd.read_csv(r"/content/drive/MyDrive/Veri/Migros/products3Yeni.csv", encoding="utf-8", sep =";")
raw_prodcuts4 = pd.read_csv(r"/content/drive/MyDrive/Veri/Migros/products4Yeni.csv", encoding="utf-8", sep =";")

raw_prodcuts5 = pd.read_csv(r"/content/drive/MyDrive/Veri/CagriMarket/csv_veri/BebekUrunleri.csv", encoding="utf-8")
raw_prodcuts6 = pd.read_csv(r"/content/drive/MyDrive/Veri/CagriMarket/csv_veri/DeterjanVeTemizlik.csv", encoding="utf-8")
raw_prodcuts7 = pd.read_csv(r"/content/drive/MyDrive/Veri/CagriMarket/csv_veri/EtTavukHindi.csv", encoding="utf-8")
raw_prodcuts8 = pd.read_csv(r"/content/drive/MyDrive/Veri/CagriMarket/csv_veri/EvVeYasamUrunleri.csv", encoding="utf-8")
raw_prodcuts9 = pd.read_csv(r"/content/drive/MyDrive/Veri/CagriMarket/csv_veri/GidaSekerleme.csv", encoding="utf-8")
raw_prodcuts10 = pd.read_csv(r"/content/drive/MyDrive/Veri/CagriMarket/csv_veri/KagitUrunleri.csv", encoding="utf-8")
raw_prodcuts11 = pd.read_csv(r"/content/drive/MyDrive/Veri/CagriMarket/csv_veri/KisiselBakimVeKozmetik.csv", encoding="utf-8")
raw_prodcuts12 = pd.read_csv(r"/content/drive/MyDrive/Veri/CagriMarket/csv_veri/MeyveSebze.csv", encoding="utf-8")
raw_prodcuts13 = pd.read_csv(r"/content/drive/MyDrive/Veri/CagriMarket/csv_veri/PetShopUrunleri.csv", encoding="utf-8")
raw_prodcuts14 = pd.read_csv(r"/content/drive/MyDrive/Veri/CagriMarket/csv_veri/SutveKahvalti.csv", encoding="utf-8")
raw_prodcuts15 = pd.read_csv(r"/content/drive/MyDrive/Veri/CagriMarket/csv_veri/icecekler.csv", encoding="utf-8")


In [ ]:
print(raw_prodcuts5["urunKategorileri"].value_counts())
print(raw_prodcuts6["urunKategorileri"].value_counts())
print(raw_prodcuts7["urunKategorileri"].value_counts())
print(raw_prodcuts8["urunKategorileri"].value_counts())
print(raw_prodcuts9["urunKategorileri"].value_counts())
print(raw_prodcuts10["urunKategorileri"].value_counts())
print(raw_prodcuts11["urunKategorileri"].value_counts())
print(raw_prodcuts12["urunKategorileri"].value_counts())
print(raw_prodcuts13["urunKategorileri"].value_counts())
print(raw_prodcuts14["urunKategorileri"].value_counts())
print(raw_prodcuts15["urunKategorileri"].value_counts())

In [ ]:
raw_prodcuts5["urunKategorileri"] = "Bebek, Oyuncak"
raw_prodcuts6["urunKategorileri"] = "Deterjan, Temizlik"
raw_prodcuts7["urunKategorileri"] = "Et, Tavuk, Balık"
raw_prodcuts8["urunKategorileri"] = "Ev, Pet"
raw_prodcuts9["urunKategorileri"] = "Gıda, Şekerleme"
raw_prodcuts10["urunKategorileri"] = "Kağıt, Kozmetik"
raw_prodcuts11["urunKategorileri"] = "Kağıt, Kozmetik"
raw_prodcuts12["urunKategorileri"] = "Meyve, Sebze"
raw_prodcuts13["urunKategorileri"] = "Ev, Pet"
raw_prodcuts14["urunKategorileri"] = "Süt, Kahvaltılık"
raw_prodcuts15["urunKategorileri"] = "İçecek"

In [ ]:
raw_prodcuts1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   uMarka                 5000 non-null   object 
 1   urunKategorileri       5000 non-null   object 
 2   urunKategorileriTwo    4999 non-null   object 
 3   urunKategorileriThree  4855 non-null   object 
 4   urunKategorileriFour   2801 non-null   object 
 5   urunKategorileriFive   4 non-null      object 
 6   urunKategorileriSix    0 non-null      float64
 7   urunKategorileriSeven  0 non-null      float64
 8   urunAdi                5000 non-null   object 
 9   urunAciklama           4996 non-null   object 
 10  urunFoto               5000 non-null   object 
 11  urunURL                4999 non-null   object 
dtypes: float64(2), object(10)
memory usage: 468.9+ KB


In [ ]:
raw_prodcuts2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   uMarka                 5000 non-null   object 
 1   urunKategorileri       5000 non-null   object 
 2   urunKategorileriTwo    5000 non-null   object 
 3   urunKategorileriThree  4812 non-null   object 
 4   urunKategorileriFour   2294 non-null   object 
 5   urunKategorileriFive   3 non-null      object 
 6   urunKategorileriSix    0 non-null      float64
 7   urunKategorileriSeven  0 non-null      float64
 8   urunAdi                4998 non-null   object 
 9   urunAciklama           3972 non-null   object 
 10  urunFoto               5000 non-null   object 
 11  urunURL                5000 non-null   object 
dtypes: float64(2), object(10)
memory usage: 468.9+ KB


In [ ]:
raw_prodcuts3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   uMarka                 5000 non-null   object 
 1   urunKategorileri       5000 non-null   object 
 2   urunKategorileriTwo    4999 non-null   object 
 3   urunKategorileriThree  4891 non-null   object 
 4   urunKategorileriFour   3395 non-null   object 
 5   urunKategorileriFive   0 non-null      float64
 6   urunKategorileriSix    0 non-null      float64
 7   urunKategorileriSeven  0 non-null      float64
 8   Unnamed: 8             0 non-null      float64
 9   urunAdi                5000 non-null   object 
 10  urunAciklama           3672 non-null   object 
 11  urunFoto               5000 non-null   object 
 12  urunURL                4999 non-null   object 
dtypes: float64(4), object(9)
memory usage: 507.9+ KB


In [ ]:
raw_prodcuts4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   uMarka                 387 non-null    object 
 1   urunKategorileri       387 non-null    object 
 2   urunKategorileriTwo    387 non-null    object 
 3   urunKategorileriThree  387 non-null    object 
 4   urunKategorileriFour   255 non-null    object 
 5   urunKategorileriFive   0 non-null      float64
 6   urunKategorileriSix    0 non-null      float64
 7   urunKategorileriSeven  0 non-null      float64
 8   urunAdi                387 non-null    object 
 9   urunAciklama           295 non-null    object 
 10  urunFoto               387 non-null    object 
 11  urunURL                387 non-null    object 
dtypes: float64(3), object(9)
memory usage: 36.4+ KB


In [ ]:
frames = [raw_prodcuts1, raw_prodcuts2, raw_prodcuts3, raw_prodcuts4, raw_prodcuts5,
          raw_prodcuts6, raw_prodcuts7, raw_prodcuts8, raw_prodcuts9, raw_prodcuts10,
          raw_prodcuts11, raw_prodcuts12, raw_prodcuts13, raw_prodcuts14, raw_prodcuts15]
raw_dataframe = shuffle(pd.concat(frames))
print(f"DataFrame size = {len(raw_dataframe)}")

DataFrame size = 18767


In [ ]:
raw_dataframe.head()
raw_dataframe.to_csv("/content/drive/MyDrive/birlesik_data.csv",index = False)

In [ ]:
raw_dataframe["description"] = raw_dataframe["uMarka"] + " " + raw_dataframe["urunAdi"]
train_df = raw_dataframe[["urunKategorileri", "description"]]

In [ ]:
train_df.head()

,urunKategorileri,description
2930,"Bebek, Oyuncak",Bepanthol Bepanthol Baby Pişik Önleyici Krem 1...
3533,İçecek,Dimes Dimes Ekşi Elma 1 L - Migros
2552,"Deterjan, Temizlik",Koroplast Koroplast Dolap İçi Çöp Torbası 10'L...
340,"Süt, Kahvaltılık",Pınar Pınar Tereyağı 1 Kg - Migros
4629,"Kağıt, Kozmetik",Elidor Elidor Anında Onarıcı Bakım Şampuanı 65...


In [ ]:
train_df.isnull().sum()

urunKategorileri     0
description         13
dtype: int64

In [ ]:
train_df = train_df.dropna()
train_df.isnull().sum()

urunKategorileri    0
description         0
dtype: int64

In [ ]:
train_df.shape

(18754, 2)

In [ ]:
train_df.head(10)

,urunKategorileri,description
2930,"Bebek, Oyuncak",Bepanthol Bepanthol Baby Pişik Önleyici Krem 1...
3533,İçecek,Dimes Dimes Ekşi Elma 1 L - Migros
2552,"Deterjan, Temizlik",Koroplast Koroplast Dolap İçi Çöp Torbası 10'L...
340,"Süt, Kahvaltılık",Pınar Pınar Tereyağı 1 Kg - Migros
4629,"Kağıt, Kozmetik",Elidor Elidor Anında Onarıcı Bakım Şampuanı 65...
1648,"Kağıt, Kozmetik",Maybelline Maybelline Super Stay Ink Kapatıcı ...
2419,"Kağıt, Kozmetik",O.B O.b. Tampon Normal 16 Adet - Migros
162,"Bebek, Oyuncak",Canbebe Canbebe Jumbo Midi 34 lü
836,"Gıda, Şekerleme",Berrak Berrak Kapari Turşusu 180 G - Migros
1287,"Bebek, Oyuncak",Bebelac Bebelac Gold 1 Bebek Sütü 900 G 0-6 Ay...


In [ ]:
train_df.tail()

,urunKategorileri,description
824,"Süt, Kahvaltılık",Teremyağ Teremyağ Gurme Kaymaklı Paket 250 G -...
4106,"Kağıt, Kozmetik",Oral-B Oral-B Pulsar Diş Eti Ve Hassas Bakım D...
161,"Ev, Pet",Züccaciye Sembol 182 Lake Çok Amaçlı Dolap 600...
4890,"Bebek, Oyuncak",Reyondan İlk Kelimelerim. - Migros
3425,"Ev, Pet",Roll-Up Roll-up Partydreams Plastik Bardak 25'...


In [ ]:
nltk.download('stopwords')
WPT = nltk.WordPunctTokenizer()
stop_words_list = nltk.corpus.stopwords.words('turkish')
#stop_words_list += ["li", "lu", "lü", "gr", "kg", "ml", "lt", "migros", "g",
#                   "cm", "m", "x", "l", "reyondan"]
stop_words_list += ["migros", "reyondan", "orgnanik", "gezen", "boy", "küçük"]
print(stop_words_list)
TAG_RE = re.compile(r'<[^>]+>')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani', 'migros', 'reyondan', 'orgnanik', 'gezen', 'boy', 'küçük']


In [ ]:
def html_totext(text):
    return html2text(text)


def remove_tags(text):
    return TAG_RE.sub(' ', text) ##HTML TAGLERİNİ SİLER

def lower_letters(txt):
    txt = txt.replace("İ", "i")
    return txt.lower()


def resub_comma(txt):
    txt= txt.replace(",", " ")
    return txt


def remove_stopwords(txt):
    return ' '.join([word for word in txt.split() if word.strip() not in stop_words_list]) ## Yukarıdaki stopwordsleri siler


def remove_punctuation(txt):
    return re.sub(r'[^\w\s]', ' ', txt) 


def remove_repeatedLetter(txt):
     return (''.join(i for i, _ in itertools.groupby(txt)))
 
    
def remove_integer(txt):
    return ''.join([word for word in txt if not word.isdigit()])


def just_one_word(txt):
    #return ' '.join(set(txt.split(" ")))
    new_txt = []
    for word in txt.split(' '):
        if word not in new_txt:
            new_txt.append(word)
    return ' '.join(new_txt)


def checkadem(txt):
    return txt + " adem"

def clean_text(txt):
    txt = html_totext(txt)
    txt = remove_tags(txt)
    txt = lower_letters(txt)
    txt = resub_comma(txt)
    txt = remove_punctuation(txt)
    txt = remove_stopwords(txt)
    txt = remove_integer(txt)
    txt = remove_repeatedLetter(txt)
    txt = just_one_word(txt)
    return txt

In [ ]:
train_df["urunKategorileri"] = train_df["urunKategorileri"].apply(str.strip)

In [ ]:
train_df["urunKategorileri"].value_counts()

Kağıt, Kozmetik       4225
Ev, Pet               3763
Gıda, Şekerleme       3497
Deterjan, Temizlik    1990
İçecek                1611
Süt, Kahvaltılık      1457
Bebek, Oyuncak        1369
Et, Tavuk, Balık       505
Meyve, Sebze           243
Elektronik              56
Shoulders               29
Guy                      6
Bugün Eklenenler         2
Oreal                    1
Name: urunKategorileri, dtype: int64

In [ ]:
train_df = train_df[~train_df["urunKategorileri"].isin(["Bugün Eklenenler", "Elektronik", "Guy", "Shoulders", "Oreal"])]

In [ ]:
train_df["description"] = train_df["description"].apply(clean_text)
train_df.head()

,urunKategorileri,description
2930,"Bebek, Oyuncak",bepanthol baby pişik önleyici krem g
3533,İçecek,dimes ekşi elma l
2552,"Deterjan, Temizlik",koroplast dolap içi çöp torbası lu
340,"Süt, Kahvaltılık",pınar tereyağı kg
4629,"Kağıt, Kozmetik",elidor anında onarıcı bakım şampuanı ml


In [ ]:
s = train_df['description'].str.split(expand=True).stack().value_counts()
train_df["urunKategorileri"].value_counts()

Kağıt, Kozmetik       4225
Ev, Pet               3763
Gıda, Şekerleme       3497
Deterjan, Temizlik    1990
İçecek                1611
Süt, Kahvaltılık      1457
Bebek, Oyuncak        1369
Et, Tavuk, Balık       505
Meyve, Sebze           243
Name: urunKategorileri, dtype: int64

In [ ]:
train_df.to_csv("datalar.csv", index = False)

In [ ]:
device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")
bert = AutoModel.from_pretrained("dbmdz/bert-base-turkish-128k-uncased").to(device)

In [ ]:
def feature_extraction(text):
    x = tokenizer.encode(text)
    with torch.no_grad():
        x, _ = bert(torch.stack([torch.tensor(x)]).to(device))
        return list(x[0][0].cpu().numpy())

In [ ]:
train, test = train_test_split(train_df, test_size=0.0001, random_state=42)
train.shape, test.shape

((18658, 2), (2, 2))

In [ ]:
train_df

,urunKategorileri,description
2930,"Bebek, Oyuncak",bepanthol baby pişik önleyici krem g
3533,İçecek,dimes ekşi elma l
2552,"Deterjan, Temizlik",koroplast dolap içi çöp torbası lu
340,"Süt, Kahvaltılık",pınar tereyağı kg
4629,"Kağıt, Kozmetik",elidor anında onarıcı bakım şampuanı ml
...,...,...
824,"Süt, Kahvaltılık",teremyağ gurme kaymaklı paket g
4106,"Kağıt, Kozmetik",oral b pulsar diş eti hasas bakım fırçası
161,"Ev, Pet",zücaciye sembol lake amaçlı dolap x
4890,"Bebek, Oyuncak",ilk kelimelerim


In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/trendyol_test_2.csv", sep =";", encoding="utf-8")
test_data = test_data[["product_category", "name"]]
test_data["name"] = test_data["name"].apply(clean_text)
test_data["product_category"] = "Süt, Kahvaltılık"
test = test_data 
test

,product_category,name
0,"Süt, Kahvaltılık",serbest tavuk yumurtası li
1,"Süt, Kahvaltılık",doğal köy yumurtası adet
2,"Süt, Kahvaltılık",tavuk koy yumurtası ad
3,"Süt, Kahvaltılık",serbest tavuk doğal köy yumurtası lı büyük gr
4,"Süt, Kahvaltılık",organik yumurta lu
...,...,...
66,"Süt, Kahvaltılık",lı renkli yumurtalık
67,"Süt, Kahvaltılık",organik yumurta lu
68,"Süt, Kahvaltılık",köy yumurtası adet
69,"Süt, Kahvaltılık",organik lu m orta yumurta g adet


In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []
for index, element in tqdm(train.iterrows()):
    X_train.append(feature_extraction(element[1]))
    y_train.append(element[0])
for index, element in tqdm(test.iterrows()):
    X_test.append(feature_extraction(element[1]))
    y_test.append(element[0])

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=1000,
                      activation='tanh', solver='adam', 
                      alpha=1e-5, learning_rate='constant',
                      verbose=1, early_stopping=True)
model.fit(X_train, y_train)
y_true, y_pred = y_test, model.predict(X_test)

Iteration 1, loss = 1.20478292
Validation score: 0.705252
Iteration 2, loss = 0.73117657
Validation score: 0.771168
Iteration 3, loss = 0.60477548
Validation score: 0.793140
Iteration 4, loss = 0.52525920
Validation score: 0.781350
Iteration 5, loss = 0.47753676
Validation score: 0.811897
Iteration 6, loss = 0.41663621
Validation score: 0.807074
Iteration 7, loss = 0.37721024
Validation score: 0.821543
Iteration 8, loss = 0.35427373
Validation score: 0.823687
Iteration 9, loss = 0.30828738
Validation score: 0.840300
Iteration 10, loss = 0.29729680
Validation score: 0.837085
Iteration 11, loss = 0.24710917
Validation score: 0.848875
Iteration 12, loss = 0.23460182
Validation score: 0.848875
Iteration 13, loss = 0.20798836
Validation score: 0.861200
Iteration 14, loss = 0.19321909
Validation score: 0.856913
Iteration 15, loss = 0.17518241
Validation score: 0.840836
Iteration 16, loss = 0.15935777
Validation score: 0.853698
Iteration 17, loss = 0.15400924
Validation score: 0.851018
Iterat

In [ ]:
print(classification_report(y_true, y_pred))

                  precision    recall  f1-score   support

  Bebek, Oyuncak       0.00      0.00      0.00         0
Et, Tavuk, Balık       0.00      0.00      0.00         0
         Ev, Pet       0.00      0.00      0.00         0
 Gıda, Şekerleme       0.00      0.00      0.00         0
 Kağıt, Kozmetik       0.00      0.00      0.00         0
    Meyve, Sebze       0.00      0.00      0.00         0
Süt, Kahvaltılık       1.00      0.32      0.49        71

        accuracy                           0.32        71
       macro avg       0.14      0.05      0.07        71
    weighted avg       1.00      0.32      0.49        71



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
"""from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier"""

'from sklearn.naive_bayes import GaussianNB\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.svm import SVC\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.discriminant_analysis import LinearDiscriminantAnalysis\nfrom sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier\nfrom sklearn.neural_network import MLPClassifier'

In [ ]:
"""models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('Naive Bayes', GaussianNB()))
models.append(('Decision Tree (CART)',DecisionTreeClassifier())) 
models.append(('K-NN', KNeighborsClassifier()))
models.append(('SVM', SVC()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('BaggingClassifier', BaggingClassifier()))
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('MLPClassifier', MLPClassifier()))

#burada bir döngü vasıtasıyla tek tek bütün modelleri deneyerek sonuçları karşılaştırıyoruz. 
for name, modeld in models:
    modeld = modeld.fit(X_train, y_train)
    y_pred = modeld.predict(X_test)
    from sklearn import metrics

    print("%s -> ACC: %%%.2f" % (name,metrics.accuracy_score(y_test, y_pred)*100))"""

'models = []\nmodels.append((\'Logistic Regression\', LogisticRegression()))\nmodels.append((\'Naive Bayes\', GaussianNB()))\nmodels.append((\'Decision Tree (CART)\',DecisionTreeClassifier())) \nmodels.append((\'K-NN\', KNeighborsClassifier()))\nmodels.append((\'SVM\', SVC()))\nmodels.append((\'AdaBoostClassifier\', AdaBoostClassifier()))\nmodels.append((\'BaggingClassifier\', BaggingClassifier()))\nmodels.append((\'RandomForestClassifier\', RandomForestClassifier()))\nmodels.append((\'MLPClassifier\', MLPClassifier()))\n\n#burada bir döngü vasıtasıyla tek tek bütün modelleri deneyerek sonuçları karşılaştırıyoruz. \nfor name, modeld in models:\n    modeld = modeld.fit(X_train, y_train)\n    y_pred = modeld.predict(X_test)\n    from sklearn import metrics\n\n    print("%s -> ACC: %%%.2f" % (name,metrics.accuracy_score(y_test, y_pred)*100))'

In [ ]:
test = "Orta boy yumurta"
test = clean_text(test)
X = feature_extraction(test)
cc = model.predict([X])
print(cc)

['Gıda, Şekerleme']


In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/trendyol_test_2.csv", sep=";", encoding="utf-8")
test_data = test_data[["product_category", "name"]]
len(test_data)

71

In [ ]:
test = "muz yerli"
test = clean_text(test)
X = feature_extraction(test)
cc = model.predict([X])
print(cc)

['Ev, Pet']


In [ ]:
wars = 0
def test_ml(txt):
    raw_txt = txt
    txt = clean_text(txt)
    X = feature_extraction(txt)
    cc = model.predict([X])[0]


    print(f"{raw_txt}  ->  {cc}")

test_data["name"].apply(test_ml)
    #10DİPNOT
print(wars)



Serbest Gezen Tavuk Yumurtası 15'li  ->  Et, Tavuk, Balık
Doğal Köy Yumurtası 30 Adet  ->  Meyve, Sebze
Gezen Tavuk Koy Yumurtası 30 Ad  ->  Süt, Kahvaltılık
Serbest Gezen Tavuk,doğal Köy Yumurtası, 60'lı Büyük Boy (63-72gr)  ->  Gıda, Şekerleme
Organik Yumurta 10 Lu  ->  Gıda, Şekerleme
30'lu Küçük Boy (44-52 gr) Organik Yumurta  ->  Süt, Kahvaltılık
60 Lı Büyük Boy (63-72 Gr) Organik Yumurta  ->  Ev, Pet
Köy Yumurtası 20 Adet  ->  Gıda, Şekerleme
Doğal Köy Yumurtası 30’lu Paket  ->  Süt, Kahvaltılık
Köy Yumurta 20 Adetx3 Karton  ->  Gıda, Şekerleme
Köy Yumuratsı Şile Ağvada Kendi Tavuklarımızdan  ->  Bebek, Oyuncak
Bıldırcın Yumurtasi (10 Paket)  ->  Ev, Pet
Porelin Yumurtalık 6 Adet  ->  Kağıt, Kozmetik
60'lı Küçük Boy (44-52 Gr) Organik Yumurta  ->  Ev, Pet
Organik Gezen Tavuk Yumurtası 30’lu Yumurta  ->  Süt, Kahvaltılık
Yumurta Köy 60 Ad Organik M Boy Ferik Yumurtası Baskılı Ciflik Yumurtası Degil El Emeği Göz Nuru  ->  Süt, Kahvaltılık
30'lu Büyük Boy (63-72gr) Gezen Tavuk Yumur

In [ ]:
#TODO: Kategori Bazında alt kategorileri öğret

In [ ]:
import pickle
handle = open("marketML.dat", "wb")
pickle.dump(model, handle)
handle.close()

# TODO:


*   Süt ve Kahvaltı Kategorisinde en çok geçen kelimeler ve adetleri
*   En fazla geçen kelime ve adetleri 
*   Diğer kategorilerde en çok geçen kelimeler ve adetleri
*   İstegelsini dataları eğitime sok
*   Yanlış bilenleri sayısı, Hangilerini yanlış bilmiş 

## linkler

*    https://medium.com/spark-nlp/classification-of-texts-written-in-turkish-language-using-spark-nlp-aa9cc2b5aaba
*   https://www.kaggle.com/ayhanc/bert-multilingual-for-turkish-text-classification


**kalın metin**#TODO

*    Farlı markaların aynı ürünlerini listeden çıkart
*    Az olan kategorideki ürünlerin isimleri artır

### Eğitim TODO:

*   Berk 128 k nedir biz ne kullanmalıyız ?
*   MLPClassifier doğruluğu attırmak için optimazyon değerleri türleri konu hakkında detaylı bilgi

### Furkan TODO:

*   Veri Birleştir
*   EDA Yap
*   Tekrar eden verileri sil
*   Az olan kategorileri çoğalta